# Experiment No. 4

### Name: Vivek Vitthal Avhad (4031)

In [1]:
# Conditional GAN (cGAN) on CIFAR-10
# Grayscale -> Color Translation

import os, time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers  #type:ignore
import matplotlib.pyplot as plt

2025-11-03 18:30:01.428646: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-03 18:30:01.536921: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-03 18:30:01.623820: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762174801.710997    8676 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762174801.734662    8676 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762174801.915755    8676 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# ------------ Config -----------------
BATCH_SIZE   = 256
EPOCHS       = 10  # Reduced from 100 to 10 for faster training
LAMBDA_L1    = 100.0
LR           = 2e-4
BETA_1       = 0.5
IMG_SIZE     = 32
CHANNELS     = 3
SAMPLE_DIR   = "cgan_cifar_samples"
os.makedirs(SAMPLE_DIR, exist_ok=True)

In [3]:
# Data
(x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

In [4]:
# Condition = grayscale version, Target = color
def rgb_to_gray(x):
    return np.mean(x, axis=-1, keepdims=True)

X_cond_train = rgb_to_gray(x_train)  # grayscale
Y_tgt_train  = x_train               # color
X_cond_test  = rgb_to_gray(x_test)
Y_tgt_test   = x_test

In [5]:
# Scale [-1,1]
def scale_neg1_1(x): return x * 2.0 - 1.0
X_cond_train = scale_neg1_1(X_cond_train)
Y_tgt_train  = scale_neg1_1(Y_tgt_train)
X_cond_test  = scale_neg1_1(X_cond_test)
Y_tgt_test   = scale_neg1_1(Y_tgt_test)

In [6]:
train_ds = tf.data.Dataset.from_tensor_slices((X_cond_train, Y_tgt_train)).shuffle(60000).batch(BATCH_SIZE)
test_ds  = tf.data.Dataset.from_tensor_slices((X_cond_test, Y_tgt_test)).batch(BATCH_SIZE)

2025-11-03 18:30:10.235768: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-11-03 18:30:10.239730: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 204800000 exceeds 10% of free system memory.
2025-11-03 18:30:11.628099: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.
2025-11-03 18:30:12.526317: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 40960000 exceeds 10% of free system memory.
2025-11-03 18:30:11.628099: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.
2025-11-03 18:30:12.526317: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 40960000 exceeds 10% of free system memory.
2025-11-03 18:30:12.667774: W external/local_xla/xla/tsl/fr

In [7]:
# Models
def down_block(filters, apply_bn=True):
    block = tf.keras.Sequential()
    block.add(layers.Conv2D(filters, 4, strides=2, padding="same", use_bias=not apply_bn))
    if apply_bn:
        block.add(layers.BatchNormalization())
    block.add(layers.LeakyReLU(0.2))
    return block

def up_block(filters, apply_dropout=False):
    block = tf.keras.Sequential()
    block.add(layers.Conv2DTranspose(filters, 4, strides=2, padding="same", use_bias=False))
    block.add(layers.BatchNormalization())
    if apply_dropout:
        block.add(layers.Dropout(0.5))
    block.add(layers.ReLU())
    return block

In [8]:
# Bigger U-Net Generator
def build_generator():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1))

    d1 = down_block(64, apply_bn=False)(inputs)   # 16x16
    d2 = down_block(128)(d1)                      # 8x8
    d3 = down_block(256)(d2)                      # 4x4
    d4 = down_block(512)(d3)                      # 2x2
    d5 = down_block(512)(d4)                      # 1x1

    b = layers.Conv2D(1024, 4, strides=1, padding="same", use_bias=False)(d5) # 1x1
    b = layers.BatchNormalization()(b)
    b = layers.ReLU()(b)

    # Upsampling layers - need to ensure they upsample to the correct size for concatenation and final output
    u1 = up_block(512)(b)                          # 2x2
    u1 = layers.Concatenate()([u1, d4])            # 2x2 + 2x2 -> 2x2

    u2 = up_block(256)(u1)                         # 4x4
    u2 = layers.Concatenate()([u2, d3])            # 4x4 + 4x4 -> 4x4

    u3 = up_block(128)(u2)                         # 8x8
    u3 = layers.Concatenate()([u3, d2])            # 8x8 + 8x8 -> 8x8

    u4 = up_block(64)(u3)                          # 16x16
    u4 = layers.Concatenate()([u4, d1])            # 16x16 + 16x16 -> 16x16

    # Add an additional upsampling layer to get to 32x32
    u5 = up_block(64)(u4)                          # 32x32
    u5 = layers.Concatenate()([u5, inputs])        # 32x32 + 32x32 -> 32x32

    out = layers.Conv2D(CHANNELS, 3, padding="same", activation="tanh")(u5) # 32x32x3
    return tf.keras.Model(inputs, out, name="Heavy_Generator")


In [9]:

# PatchGAN Discriminator
def build_discriminator():
    inp = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1))  # grayscale
    tar = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))  # color
    x = layers.Concatenate(axis=3)([inp, tar])         # 32x32x4

    x = layers.Conv2D(64, 4, strides=2, padding="same")(x) # 16x16
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(128, 4, strides=2, padding="same", use_bias=False)(x) # 8x8
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(256, 4, strides=2, padding="same", use_bias=False)(x) # 4x4
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(512, 4, strides=1, padding="same", use_bias=False)(x) # 4x4
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    logits = layers.Conv2D(1, 4, strides=1, padding="same")(x) # 4x4 Patch output
    return tf.keras.Model([inp, tar], logits, name="Heavy_Discriminator")


In [10]:
G = build_generator()
D = build_discriminator()

2025-11-03 18:30:12.950091: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16777216 exceeds 10% of free system memory.


In [11]:
# Loss & Optims
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
g_opt = tf.keras.optimizers.Adam(LR, beta_1=BETA_1)
d_opt = tf.keras.optimizers.Adam(LR, beta_1=BETA_1)

In [12]:
def discriminator_loss(d_real, d_fake):
    real_loss = bce(tf.ones_like(d_real), d_real)
    fake_loss = bce(tf.zeros_like(d_fake), d_fake)
    return real_loss + fake_loss

def generator_loss(d_fake, real, fake):
    adv = bce(tf.ones_like(d_fake), d_fake)
    l1 = tf.reduce_mean(tf.abs(real - fake))
    return adv + LAMBDA_L1 * l1, adv, l1

@tf.function
def train_step(x_cond, y_tgt):
    with tf.GradientTape(persistent=True) as tape:
        y_fake = G(x_cond, training=True)

        d_real = D([x_cond, y_tgt], training=True)
        d_fake = D([x_cond, y_fake], training=True)

        d_loss = discriminator_loss(d_real, d_fake)
        g_total, g_adv, g_l1 = generator_loss(d_fake, y_tgt, y_fake)

    d_grads = tape.gradient(d_loss, D.trainable_variables)
    g_grads = tape.gradient(g_total, G.trainable_variables)

    d_opt.apply_gradients(zip(d_grads, D.trainable_variables))
    g_opt.apply_gradients(zip(g_grads, G.trainable_variables))

    return d_loss, g_total, g_adv, g_l1

In [13]:
# Visualization
def show_samples(epoch):
    x_batch, y_batch = next(iter(test_ds))
    y_pred = G(x_batch, training=False)
    def to01(t): return (t + 1.0) / 2.0
    x = to01(x_batch).numpy()
    y = to01(y_batch).numpy()
    p = to01(y_pred).numpy()

    n = 6
    plt.figure(figsize=(12, 6))
    for i in range(n):
        ax = plt.subplot(3, n, i+1)
        plt.imshow(x[i,...,0], cmap="gray")
        ax.set_title("Gray")
        ax.axis("off")

        ax = plt.subplot(3, n, i+1+n)
        plt.imshow(y[i])
        ax.set_title("Real Color")
        ax.axis("off")

        ax = plt.subplot(3, n, i+1+2*n)
        plt.imshow(p[i])
        ax.set_title("Fake Color")
        ax.axis("off")

    plt.suptitle(f"Epoch {epoch}")
    path = os.path.join(SAMPLE_DIR, f"epoch_{epoch:03d}.png")
    plt.savefig(path)
    plt.show()

In [ ]:
# Training Loop
for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    d_losses, g_losses = [], []
    for x_cond, y_tgt in train_ds:
        d_loss, g_total, g_adv, g_l1 = train_step(x_cond, y_tgt)
        d_losses.append(d_loss.numpy())
        g_losses.append(g_total.numpy())

    print(f"Epoch {epoch}/{EPOCHS} | D: {np.mean(d_losses):.4f} | G: {np.mean(g_losses):.4f} | Time: {time.time()-t0:.1f}s")

    if epoch % 2 == 0 or epoch == 1:  # Show samples every 2 epochs instead of 5
        show_samples(epoch)